# Selection question: AlexNet

## Implementation and optimization of a convolutional neural network

Implement a classic convolutional neural network: AlexNet. **Improve its efficiency as much as possible.** Your work will be evaluated by four aspects: *performance, algorithm design, software architecture and readability.*

- Neural network should be implemented by hand, open source arch(Caffe, Tensorflow, Theano, Torch) should not be used directly.
- Both inference and train algorithm should be constructed, performance should be well considered either.
- Project deadline: 2016-12-25

## Getting started

To achieve a easy start, I copied some useful code from https://www.zybuluo.com/hanbingtao/note/476663 and https://www.zybuluo.com/hanbingtao/note/485480, code list:
- class Layer, class Node, class ConstNode, class Connection
- class ConvLayer

With the codes above, we have the ability to:
- Construct a full connection layer instance with class Layer, forward calculation function not included in those classes
- Initialize a convolutional layer with class ConvLayer, including backpropagation algorithm and forward computation method

To achieve the goal mentioned at beginning, there exists several works to be finished:
- Design a network class, composing convolutional layer and full connection layer together. Both train and predict function should be implemented. 
- Optimize the training and predicting procedure, improve performance of the network. 

## Software architecture

### Inversion of Control

Before we design the network class, it might be wise to take a look at **alexnet.py** in tensorflow:

In [ ]:
net = slim.conv2d(inputs, 64, [11, 11], 4, padding='VALID',scope='conv1')
net = slim.max_pool2d(net, [3, 3], 2, scope='pool1')
net = slim.conv2d(net, 192, [5, 5], scope='conv2')
net = slim.max_pool2d(net, [3, 3], 2, scope='pool2')
net = slim.conv2d(net, 384, [3, 3], scope='conv3')
net = slim.conv2d(net, 384, [3, 3], scope='conv4')
net = slim.conv2d(net, 256, [3, 3], scope='conv5')
net = slim.max_pool2d(net, [3, 3], 2, scope='pool5')

## Performance